In [ ]:
!pip install paho-mqtt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import paho.mqtt.client as mqtt
client=mqtt.Client()
client.username_pw_set(username='mirae',password='daejeon1234')
client.connect('34.64.69.204',19883)
client.publish('house/light','빨리켜져라')

In [ ]:
client.subscribe('house/light',qos=0)

(0, 2)

In [ ]:
import json

def on_connect(client,userdata,flags,rc):
  print('연결되었습니다. '+str(rc))
def on_message(client,userdata,msg):
  print(msg.topic+" "+str(msg.payload))

  try:
    json_data = json.loads(msg.payload)
    print("Parsed JSON data: " + str(json_data))
  except ValueError:
    print("Received message is not valid JSON")

In [ ]:
#이벤트처리
client=mqtt.Client()
# client.username_pw_set(username='testadmin',password="")
# 웹에서 event처리할 때
client.on_connect=on_connect
client.on_message=on_message
client.connect('34.64.69.204',19883)
client.subscribe('house/light',qos=1)
client.loop_forever()

연결되었습니다. 0
house/light b'\xeb\xb9\xa8\xeb\xa6\xac\xec\xbc\x9c\xec\xa0\xb8\xeb\x9d\xbc'
Received message is not valid JSON
house/light b'quick'
Received message is not valid JSON
house/light b'light on'
Received message is not valid JSON
house/light b'{"light": "turnoff", "door": "close"}'
Parsed JSON data: {'light': 'turnoff', 'door': 'close'}


KeyboardInterrupt: ignored

In [ ]:
# JSON 수신 프로그램

In [ ]:
import json
def on_connect(client,userdata,flags,rc):
  print('연결되었습니다. '+str(rc))
# def on_message(client,userdata,msg):
#   json_return=json.loads(msg.payload.decode('utf-8'))
#   print(json_return['light'])
#   print(msg.topic+' '+str(json_return))
def on_message(client, userdata, msg):
    try:
        json_return = json.loads(msg.payload)
        print(json_return['light'])
        print(msg.topic + ' ' + str(json_return))
    except json.JSONDecodeError as e:
        print("Received message is not valid JSON:", e)

In [ ]:
client=mqtt.Client()
client.username_pw_set(username='daejeon',password='daejeon1234')
client.on_connect=on_connect
client.on_message=on_message
client.connect('34.64.69.204',19883)
client.subscribe('house/light',qos=1)
client.loop_forever()

연결되었습니다. 0
Received message is not valid JSON: Expecting value: line 1 column 1 (char 0)
on
house/light {'light': 'on', 'door': 'close'}


KeyboardInterrupt: ignored

In [18]:
import mysql.connector

# MariaDB 연결 정보
db_host = '34.64.69.204'
db_user = 'root'
db_password = 'acorn1234'
db_name = 'iot'

# MariaDB 연결
conn = mysql.connector.connect(host=db_host, user=db_user, password=db_password, database=db_name)
cursor = conn.cursor()

# ALTER TABLE 문 실행
alter_query = "ALTER TABLE sensorsignal ADD COLUMN light VARCHAR(50), ADD COLUMN door VARCHAR(50),ADD COLUMN close VARCHAR(50), ADD COLUMN hangeul VARCHAR(50)"
cursor.execute(alter_query)

# 변경사항 커밋
conn.commit()

# 연결 종료
cursor.close()
conn.close()

print("Columns added successfully to the table.")


Columns added successfully to the table.


In [ ]:
# 문제
# 수신한 데이터를 mariadb 테이블에 저장하시오
# publish되면 subscribe해서 mariadb에 저장 

In [ ]:
!pip install mysql-connector-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import paho.mqtt.client as mqtt
import json
import mysql.connector

# MariaDB 연결 정보
db_host = '34.64.69.204'
db_user = 'root'
db_password = 'acorn1234'
db_name = 'mqtt'
# MariaDB 연결
conn = mysql.connector.connect(host=db_host, user=db_user, password=db_password, database=db_name)
cursor = conn.cursor()

def on_connect(client, userdata, flags, rc):
    print('연결되었습니다. ' + str(rc))

def on_message(client, userdata, msg):
  json_data = json.loads(msg.payload)
  try:
    with conn.cursor() as cursor:
      # 데이터 삽입 쿼리
      query = "INSERT INTO jsptest (light, door) VALUES (%s, %s)"
      values = (json_data['light'], json_data['door'])

      # 쿼리 실행
      cursor.execute(query, values)
    conn.commit()

    # 연결 종료
    cursor.close()
    conn.close()

    print("데이터를 MariaDB에 저장했습니다.")
  except json.JSONDecodeError as e:

    print("Received message is not valid JSON:", e)


In [16]:
client = mqtt.Client()
client.username_pw_set(username='daejeon', password='daejeon1234')
client.on_connect = on_connect
client.on_message = on_message
client.connect('34.64.69.204', 19883)
client.subscribe('house/light', qos=1)
client.loop_forever()


연결되었습니다. 0
데이터를 MariaDB에 저장했습니다.


KeyboardInterrupt: ignored

In [ ]:
import paho.mqtt.client as mqtt
import json
import mysql.connector

# MariaDB 연결 정보
db_host = '34.64.69.204'
db_user = 'root'
db_password = 'acorn1234'
db_name = 'iot'
# MariaDB 연결
conn = mysql.connector.connect(host=db_host, user=db_user, password=db_password, database=db_name)
def on_connect(client, userdata, flags, rc):
    print('연결되었습니다. ' + str(rc))
    client.subscribe('house/light')
def on_message(client, userdata, msg):
  json_return = json.loads(msg.payload.decode('utf-8'))
  print(json_return['light'])
  print(json_return['hangeul'])
  print(msg.topic+' '+str(json_return))
  try:
    with conn.cursor() as cursor:
      sql='INSERT INTO sensorsignal (light,door,close,hangeul) VALUES (%s, %s, %s, %s)'
      cursor.execute(sql,(json_return['light'],json_return['door'],json_return['close'],json_return['hangeul']))
      conn.commit()
      print('저장완료')
  except Exception as e:
    print('실행오류:',e)
  if json_return['close']=='true':
    client.disconnect()

def on_disconnect(client,userdata,rc):
  print('해제 이유'+str(rc))
  conn.close()
  client.connected_flag=False
  client.disconnect_flag=True
def onSubscribed(client, userdata,topic):
  print('다음 토픽에 등록됨 : $topic')
def on_subscribe(client,userdata,mid,granted_qos):
  print('subscribed'+str(mid)+' '+str(granted_qos))
def on_close(client,userdata,rc):
  print('종료')
  conn.close()